## Overview

Getting the relevant data to code basis can be the bottleneck, as opposed to the actual code itself.

When this is the case, the program is calles I/O bound -> the speed is bounded by the efficiency of the input/output

Every time code reads from a file or writes to a network socket, it must pause to contact the kernel, request that the actual read happens, and then wait for it to complete.

This is because it's not the program but the kernel that does the actual read operation, as the kernel is responsible for managing any interaction with the hardware.



Most I/O operations on devices are multiple times slower than the CPU. So even if the communication with the kernel is fast, most time is spent for the kernel to get the result from the device and send it back to the program.

During this time the propram is halted, until it gets a signal that the write operation has completed. Time spent in a paused state is called I/O wait

Asynchronous I/O helps to utilize this waiting time by allowing  to perform other operations while the program is in I/O state:

![](./pics/IMG_6462.jpg)


This is possible because while a program is in I/O wait, the kernel is simply waiting for the device requested to read from to send a signal that the requested data is ready.


Instead of waiting, we can create a mechanism (event loop) so that we can dispatch requests of data, continue performing compute operations, and be notified when the data is ready to be read.

**Important: this is all still happening an a single thread and still uses one CPU at a time**

In contrast to multiprocessing/multithreading, where a new process is launched that does experience I/O wait but uses multitasking nature of modern CPUs to allow the main process to continue.

These two mechanisms are often used in tandem: multiple processes are launched, each is efficient at asynchronous I/O.


When a programm enters I/O wait, the execution is paused so that the kernel can perform low-level operations associated with I/O request (context switch), and it is not resumed until I/O operation is completed.

Context switch is a heavy operation as it requires to save the state of the program and give up the use of the CPU.

Later the program needs to be reinitialized and resumed.

With concurrency a event loop is running that manages what gets to run in the program and when.

*Event loop = list of functions that need to be run*

![](./pics/event-loop.png)



In [9]:
from queue import Queue
from functools import partial
from time import sleep

eventloop = None

class EventLoop(Queue):
    def start(self):
        for i in range(4):
            function = self.get()
            function()

def do_hello():
    global eventloop
    print("Hello")
    sleep(1)
    eventloop.put(do_world)

def do_world():
    global eventloop
    print("world")
    sleep(1)
    eventloop.put(do_hello)

eventloop = EventLoop()
eventloop.put(do_hello)
eventloop.start()

Hello
world
Hello
world


The eventloop.put(do_world) approximates an asynchronous call to the do_world function.

This operation is called **nonblocking**  -> it will return immediately but guarantee that do_world is called at some point later.


## How does async/await work?

An async function (assigned with **async def**) is called coroutine and is comparable to a generator.

An **await** statement is similar in function to a yield statement: the execution of the current function gets paused while other code is run.

Once the await / yield resolves with data, the function is resumed. 



## General syntax:

In [2]:
import asyncio

async def hello(): # <-Define coroutine with async
    print('Hello world!')
    await asyncio.sleep(1) # <- await execution of nunblocking function/coroutine
    return 'Hello again!' # <- return statement after stuff is awaited


#asyncio.run(hello()) # <- start the event loop and run the coroutine.


#### A coroutine (**async def**) must always: 
- be **await**ed, 
- executed with **asyncio.run()**, 
- scheduled with **asyncio.create_task()** or 
- gathered with **asyncio.gather()**

In [ ]:
import asyncio
import time

async def say_something(delay, words): #<- define coroutine
    print(f"Before {words}")
    await asyncio.sleep(delay)  #<- await nonblocking function/coroutine
    print(f"After {words}")

async def main():
    print(f"Started: {time.strftime('%X')}")
    task1 = asyncio.create_task(say_something(1, "Task 1")) #<- Create concurrent tasks independent from each other
    task2 = asyncio.create_task(say_something(2, "Task 2")) #<- Create concurrent tasks independent from each other
    await task1 #<- wait for task to finish
    await task2 #<- wait for task to finish
    print(f"Finished: {time.strftime('%X')}")
# asyncio.run(main())   #<- Run everything concurrently

**asyncio.gather()** takes coroutines as arguments and runs them concurrently 

-> no need to define and await each task

In [4]:
import asyncio
import time

async def greetings():  #<- define coroutine
    print("Welcome")
    await asyncio.sleep(1)  #<- await nonblocking function/coroutine
    print("Goodbye")

async def main():
    start = time.time()
    await asyncio.gather(greetings(), greetings()) #<- Await & Create multiple concurrent tasks
    elapsed = time.time() - start
    print(f"{__name__} executed in {elapsed:0.2f} seconds.")

#asyncio.run(main())

## Awaitable objects

An object is called awaitable if it can be used with the await keyword:
- coroutines

    **await asyncio.sleep(delay)**
- tasks

    task2 = asyncio.create_task(say_something(2, "Task 2"))
    
    **await task1**
- futures

    A Future is a low-level awaitable object that represents the result of an asynchronous computation.

In [5]:
from asyncio import Future
future = Future()
print(future.done())
print(future.cancelled())
future.cancel()
print(future.done())
print(future.cancelled())

False
False
True
True


## Sephamores

A sephamore works by making sure that only a certain number of coroutines can enter the context block at a time.


In [21]:
import aiohttp


def chunked_http_client(num_chunks):
    """
    Returns a function that can fetch from a URL, ensuring that only
    "num_chunks" of simultaneous connects are made.
    """
    semaphore = asyncio.Semaphore(num_chunks)  # <- define limit of requests

    async def http_get(url, client_session):  # <- new coroutine to download files files asynchonously using semaphore
        nonlocal semaphore #<- nonlocal keyword is used to work with variables inside nested functions, where the variable should not belong to the inner function.
        async with semaphore:
            async with client_session.request("GET", url) as response:
                return await response.content.read() #<- return futures

    return http_get





async def run():
    urls = ["https://binaryjazz.us/wp-json/genrenator/v1/genre/"]*1000 #<- input lists to get from
    responses = [] #<- putput list of http responses

    http_client = chunked_http_client(100) #<- set async http client wit limit of parallel requests

    async with aiohttp.ClientSession() as client_session: #<- use aiohttp.ClientSession to call nonblocking get function in http_get()
        tasks = [http_client(url, client_session) for url in urls]  # save returned futures as list
        for future in asyncio.as_completed(tasks):  # wait for futures to complete
            data = await future
            responses.append(data)
    print(len(responses))
    return responses

if __name__ =="__main__":
    loop = asyncio.get_event_loop()
    responses = loop.run_until_complete(run())


## Timeouts

asyncio.wait_for(awaitable, timeout, *) to set a timeout for an awaitable object to complete.

Use it to raise an exception if the awaitable object takes too long to complete. The exception as asyncio.TimeoutError.

In [7]:
import asyncio
async def slow_operation():
    await asyncio.sleep(400) #<- too long for timeout
    print("Completed.")

async def main():
    try:
        await asyncio.wait_for(slow_operation(), timeout=1.0)
    except asyncio.TimeoutError:
        print("Timed out!")
#asyncio.run(main())

#### Links
https://testdriven.io/blog/concurrency-parallelism-asyncio/#asyncio

https://www.integralist.co.uk/posts/python-asyncio/


https://medium.com/the-brainwave/intro-to-asynchronous-programming-in-python-with-async-io-7cb4717cd91d
